In [ ]:
# install/import quantum convolutional layer
!pip install git+https://github.com/qdevpsi3/qml-qcnn.git
import qcnn

# install/import pytorch lightning package
!pip install pytorch_lightning
import pytorch_lightning as pl

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from pytorch_lightning.metrics.functional import accuracy

In [ ]:
import warnings

warnings.filterwarnings('ignore')

# Experiment : Quantum Algorithms for Deep Convolutional Neural Networks

First, we build the dataloader and create the model using *PyTorch Lightning*

## Data Loaders
---------

In [ ]:
def get_loader(is_train,
               batch_size,
               folder='./',
               dataset='mnist',
               preprocess=((0.1307,), (0.3081,))):
    assert dataset in ['mnist', 'fashion'
                      ], "dataset can take values in ['mnist','fashion']"
    if dataset == 'mnist':
        dataset_class = torchvision.datasets.MNIST
    elif dataset == 'fashion':
        dataset_class = torchvision.datasets.FashionMNIST
    if preprocess is not None:
        transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(*preprocess)
        ])
    else:
        transform = torchvision.transforms.ToTensor()
    dataset = dataset_class('{}/{}'.format(folder, dataset),
                            train=is_train,
                            download=True,
                            transform=transform)
    loader = torch.utils.data.DataLoader(dataset,
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=2)
    return loader

## Model

In [ ]:
from qcnn import QuantumConv2d  # import convolutional layer

class QuantumConvNet(pl.LightningModule):

    def __init__(self, eps, cap, ratio, delta, lr, momentum, batch_size):
        super(QuantumConvNet, self).__init__()
        self.save_hyperparameters()
        self.build_q_nn()

    def build_q_nn(self):

        # get quantum hyperparameters
        eps = self.hparams.eps
        cap = self.hparams.cap
        ratio = self.hparams.ratio
        delta = self.hparams.delta

        # build neural network classifier
        self.qonv1 = QuantumConv2d(1, 5, 7, eps, cap, ratio, delta, stride=1)
        self.qonv2 = QuantumConv2d(5, 10, 7, eps, cap, ratio, delta, stride=1)
        self.fc1 = nn.Linear(2560, 300)
        self.fc2 = nn.Linear(300, 50)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.qonv1(x)
        x = self.qonv2(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=0)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        lr, momentum = self.hparams.lr, self.hparams.momentum
        optimizer = optim.SGD(self.parameters(), lr, momentum)
        return optimizer

    def train_dataloader(self):
        batch_size = self.hparams.batch_size
        return get_loader(is_train=True, batch_size=batch_size)

    def val_dataloader(self):
        batch_size = self.hparams.batch_size
        return get_loader(is_train=False, batch_size=batch_size)

## Hyperparameters
-----------

In [ ]:
eps = 0.01
cap = 10.
ratio = 0.5
delta = 0.01
lr = 0.01
momentum = 0.5
batch_size = 64

## Train the model
--------

In [ ]:
model = QuantumConvNet(eps, cap, ratio, delta, lr, momentum, batch_size)
use_gpu = int(torch.cuda.is_available())
trainer = pl.Trainer(gpus=use_gpu, max_epochs=3, progress_bar_refresh_rate=20)
trainer.fit(model)

In [ ]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/